In [1]:
from PIL import Image, ImageDraw, ImageChops
import os
import random
import colorsys
import argparse
import pandas as pd
from io import BytesIO
import base64
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output, no_update, dash_table
from dash.dependencies import Input, Output
import pandas as pd



In [2]:
def random_point(image_size_px: int, padding: int):
    return random.randint(padding, image_size_px - padding)


def random_color():

    # I want a bright, vivid color, so max V and S and only randomize HUE.
    h = random.random()
    s = 1
    v = 1
    float_rbg = colorsys.hsv_to_rgb(h, s, v)

    # Return as integer RGB.
    return (
        int(float_rbg[0] * 255),
        int(float_rbg[1] * 255),
        int(float_rbg[2] * 255),
    )


def interpolate(start_color, end_color, factor: float):
    # Find the color that is exactly factor (0.0 - 1.0) between the two colors.
    new_color_rgb = []
    for i in range(3):
        new_color_value = factor * end_color[i] + (1 - factor) * start_color[i]
        new_color_rgb.append(int(new_color_value))

    return tuple(new_color_rgb)


def generate_art():
    # Set size parameters.
    rescale = 2
    image_size_px = 128 * rescale
    padding = 12 * rescale

    # Create the directory and base image.
    #os.makedirs(output_dir, exist_ok=True)
    bg_color = (0, 0, 0)
    image = Image.new("RGB", (image_size_px, image_size_px), bg_color)

    # How many lines do we want to draw?
    num_lines = 10
    points = []

    # Pick the colors.
    start_color = random_color()
    end_color = random_color()

    # Generate points to draw.
    for _ in range(num_lines):
        point = (
            random_point(image_size_px, padding),
            random_point(image_size_px, padding),
        )
        points.append(point)

    # Center image.
    # Find the bounding box.
    min_x = min([p[0] for p in points])
    max_x = max([p[0] for p in points])
    min_y = min([p[1] for p in points])
    max_y = max([p[1] for p in points])

    # Find offsets.
    x_offset = (min_x - padding) - (image_size_px - padding - max_x)
    y_offset = (min_y - padding) - (image_size_px - padding - max_y)

    # Move all points by offset.
    for i, point in enumerate(points):
        points[i] = (point[0] - x_offset // 2, point[1] - y_offset // 2)

    # Draw the points.
    current_thickness = 1 * rescale
    n_points = len(points) - 1
    for i, point in enumerate(points):

        # Create the overlay.
        overlay_image = Image.new("RGB", (image_size_px, image_size_px), (0, 0, 0))
        overlay_draw = ImageDraw.Draw(overlay_image)

        if i == n_points:
            # Connect the last point back to the first.
            next_point = points[0]
        else:
            # Otherwise connect it to the next element.
            next_point = points[i + 1]

        # Find the right color.
        factor = i / n_points
        line_color = interpolate(start_color, end_color, factor=factor)

        # Draw the line.
        overlay_draw.line([point, next_point], fill=line_color, width=current_thickness)

        # Increase the thickness.
        current_thickness += rescale

        # Add the overlay channel.
        image = ImageChops.add(image, overlay_image)

    # Image is done! Now resize it to be smooth.
    image = image.resize(
        (image_size_px // rescale, image_size_px // rescale), resample=Image.ANTIALIAS
    )

    # Save the image.
    #image.save(image_path)
    return pil_to_b64(image)

def pil_to_b64(im, enc_format="png", **kwargs):
    """
    Converts a PIL Image into base64 string for HTML displaying
    :param im: PIL Image object
    :param enc_format: The image format for displaying. If saved the image will have that extension.
    :return: base64 encoding
    """

    buff = BytesIO()
    im.save(buff, format=enc_format, **kwargs)
    encoded = base64.b64encode(buff.getvalue()).decode("utf-8")

    return encoded

In [6]:
# This function generates nft images on the basis of caching
# Optionally we can choose to save the images also
def create_dash_app_final(number_of_iterations):
    df=pd.DataFrame()
    df[' index'] = range(1, number_of_iterations + 1)
    global cache_dict
    cache_dict = {}
    
    app = JupyterDash(__name__)

    server = app.server
    
    PAGE_SIZE = 10

    app.layout = html.Div([
        dash_table.DataTable(
            id='datatable-paging-page-count',
            columns=[
                {"name": i, "id": i} for i in sorted(df.columns)
            ],
            page_current=0,
            page_size=PAGE_SIZE,
            page_action='custom'
        ),
        html.Br(),
        dcc.Checklist(
            id='datatable-use-page-count',
            options=[
                {'label': 'Use page_count', 'value': 'True'}
            ],
            value=['True']
        ),
        'Page count: ',
        dcc.Input(
            id='datatable-page-count',
            type='number',
            min=1,
            max=int(number_of_iterations/PAGE_SIZE),
            value=1
        ),
        html.Div(
            [
                html.Div(id="nft-container", children=[]),
            ]
        )
    ])


    @app.callback(
        Output('datatable-paging-page-count', 'data'),
        Output('nft-container', 'children'),
        Input('datatable-paging-page-count', "page_current"),
        Input('datatable-paging-page-count', "page_size"))
    def update_table(page_current,page_size):
        temp_list = []
        
        for i in range(page_current*page_size,(page_current+ 1)*page_size):
            nft_art_image = generate_art()
            if i not in cache_dict:
                cache_dict[i] = nft_art_image
            temp_list.append(cache_dict[i])
        children=[
            html.Img(id=f"my-img-{i}",className="image", src="data:image/png;base64, " + pil_img,
                    style={"margin":"25px 50px 75px 100px", "border": "1px solid green"})
                    
                for i,pil_img in enumerate(temp_list)
            ]
        
        return df.iloc[
            page_current*page_size:(page_current+ 1)*page_size
        ].to_dict('records'),children

    @app.callback(
        Output('datatable-paging-page-count', 'page_count'),
        Input('datatable-use-page-count', 'value'),
        Input('datatable-page-count', 'value'))
    def update_table(use_page_count, page_count_value):
        if len(use_page_count) == 0 or page_count_value is None:
            return None
        return page_count_value
    return app

In [7]:
# Lets touch that million of images generation 
number_of_iterations = 1000000
app = create_dash_app_final(number_of_iterations)

In [8]:
app.run_server(mode="external")

Dash app running on http://127.0.0.1:8050/
